# INTRODUCCIÓN A LA ROBÓTICA CON AITK


### Instalación librerias

**pip install aitk.robots**

En entornos CONDA:   

**jupyter labextension install @jupyter-widgets/jupyterlab-manager**

Sin entornos CONDA:   

**jupyter nbextension enable --py widgetsnbextension**

In [2]:
import aitk.robots as bots
world = bots.World(width=150, height=40)
robot = bots.Scribbler(x=20, y=25, a=90)
world.add_robot(robot)
robot.speak('Hello, world!')
world.watch()

Random seed set to: 2525831


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

##  DEMO

In [3]:
from aitk.robots import World, Scribbler, RangeSensor, LightSensor, SmellSensor, Camera


### Un mundo con paredes, una bombilla y comida.   

Hay una fuente de comida situada cerca del origen del mundo detrás de una pequeña pared azul en ángulo.   
Hay una fuente de luz cerca del centro del mundo escondida en la esquina de las paredes cian y naranja.

In [4]:
world = World(width=200, height=150, scale=5.0)
world.add_wall("cyan", 80, 50, 90, 150)
world.add_wall("orange", 90, 50, 110, 60)
world.add_wall("blue", 0, 35, 25, 30, box=False)
world.add_bulb("yellow", 100, 70, 0, 75.0)
world.add_food(15, 10, 50)

Random seed set to: 4727860


## El robot rojo utiliza sensores IR
El robot rojo utiliza sensores de infrarrojos para detectar y evitar obstáculos. Puede determinar hacia dónde girar en función de si el obstáculo está más cerca de su parte delantera izquierda o derecha.

In [5]:
robot1 = Scribbler(x=150, y=100, a=35, color="red", name="red")
robot1.add_device(RangeSensor(position=(6,-6),width=57.3,max=20,name="left-ir"))
robot1.add_device(RangeSensor(position=(6,6),width=57.3,max=20,name="right-ir"))
world.add_robot(robot1)

## El robot rosa tiene sensores de olor y una cámara.   
El robot rosa utiliza sensores olfativos para encontrar comida. También tiene una cámara que puede ver las características del mundo. Puede girarse hacia la comida detectada a izquierda o derecha.

In [6]:
robot2 = Scribbler(x=40, y=130, a=75, color="pink", name="pink")
robot2.state["timer"] = 0 #use state variable to repeatedly reverse
robot2.add_device(Camera())
robot2.add_device(SmellSensor(position=(6,-6),name="left-smell"))
robot2.add_device(SmellSensor(position=(6,6),name="right-smell"))
world.add_robot(robot2)

## El robot amarillo utiliza sensores de luz
El robot amarillo utiliza sensores de luz para encontrar una fuente luminosa. Puede girar hacia la luz detectada a izquierda o derecha.

In [7]:
robot3 = Scribbler(x=60, y=30, a=0, color="yellow", name="yellow")
robot3.add_device(LightSensor(position=(6,-6), name="left-light"))
robot3.add_device(LightSensor(position=(6,6), name="right-light"))
world.add_robot(robot3)

## Vista del MUNDO

In [8]:
world.watch(width="700px")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

## Vista de la cámara del robot rosa
La siguiente celda de código mostrará cómo se ve el mundo a través de la cámara del robot rosa.

In [9]:
robot2['camera'].watch()

HTML(value='<style>img.pixelated {image-rendering: pixelated;}</style>')

HTML(value='<style>img.pixelated {image-rendering: pixelated;}</style>')

HTML(value='<style>img.pixelated {image-rendering: pixelated;}</style>')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00@\x00\x00\x00 \x08\x06\x00\x00\x00\xa2\x9d~\x84\x…

In [10]:
from random import random

## Comportamiento del robot rojo
El robot rojo deambula por el mundo evitando los obstáculos que encuentra. Siempre avanza, pero elige cantidades de rotación aleatorias cuando sus sensores de alcance frontal están despejados.

In [18]:
def wander_ir(robot):
    max_val = robot["left-ir"].get_max()
    if robot["left-ir"].get_distance() < max_val:
        robot.move(0.1, -0.3)
    elif robot["right-ir"].get_distance() < max_val:
        robot.move(0.1, 0.3)
    else:
        robot.move(1, random()-0.5)

## Comportamiento del robot rosa
El robot rosa deambula por el mundo buscando comida y evitando obstáculos dando marcha atrás cuando se queda parado. Utiliza una variable de estado llamada temporizador para asegurarse de que, incluso en presencia de un olor, siempre da al menos cinco pasos hacia atrás cuando encuentra un obstáculo. Si se acerca lo suficiente a la comida, intenta comérsela y, si lo consigue, termina la carrera devolviendo True. De vez en cuando elige un nuevo movimiento aleatorio. Si no se activa ninguno de sus casos de prueba, repite el movimiento anterior.

In [19]:
def search_food(robot):
    left_smell = robot["left-smell"].get_reading()
    right_smell = robot["right-smell"].get_reading()
    diff = left_smell - right_smell
    if left_smell+right_smell>1.95 and robot.eat():
        robot.move(0,0)
        robot.speak("Comida localizada!")
        return True
    if robot.state["timer"] > 5:
        robot.state["timer"] = 0
    if robot.stalled:
        robot.state["timer"] = 1
        robot.reverse()
    elif robot.state["timer"] > 0:
        robot.state["timer"] += 1
    elif diff > 0.03:
        robot.move(1, 0.3)
    elif diff < -0.03:
        robot.move(1, -0.3)
    elif world.time%2 == 0:
        robot.move(1.0, random()*0.5-0.25)
    else:
        pass #continue doing previous action

## Comportamiento del robot amarillo.
El objetivo del robot amarillo es encontrar y acercarse a una fuente de luz. Cuando no detecta luz, el robot deambula. Cuando detecta más luz a un lado, gira hacia ella. Si el robot se detiene, invierte su movimiento. De vez en cuando genera un nuevo movimiento aleatorio de deambulación. Cuando no se activa ninguna de las pruebas, el robot sigue realizando el movimiento anterior. Cuando las lecturas de luz superan un umbral, se detiene e informa de que ha encontrado la luz. Al devolver True finaliza la ejecución.

In [20]:
def search_light(robot):
    left_light = robot["left-light"].get_brightness()
    right_light = robot["right-light"].get_brightness()
    diff = left_light - right_light
    if left_light + right_light > 1.9:
        robot.move(0,0)
        robot.speak("Luz localizada!")
        return True
    if robot.stalled:
        robot.reverse()
    elif diff > 0.01:
        robot.move(1,0.5)
    elif diff < -0.01:
        robot.move(1,-0.5)   
    elif world.time%2 == 0:
        robot.move(1.0, random()*0.5-0.25)
    else:
        pass #continue doing previous action

## Simular todos los robots simultáneamente
El siguiente bloque de código reinicia el mundo, y luego pone en marcha los tres robots con sus controladores únicos. La simulación termina si el robot rosa encuentra y se come la comida, o si el robot amarillo se acerca lo suficiente a la fuente de luz, o si han transcurrido 40 segundos de tiempo real. Vuelve a la vista del mundo para ver todos los robots en acción.

Se puede volver a ejecutar este bloque de código tantas veces como se quiera y observar los resultados diferentes cada vez.

In [21]:
world.reset()
robot2.state["timer"] = 0 #reset the state variable
world.set_seed(random())
world.seconds(40,[wander_ir, search_food, search_light],real_time=True)

Using random seed: 0.10092872044002865
Using random seed: 0.12995348688337005


  0%|          | 0/400 [00:00<?, ?it/s]

Simulation stopped at: 00:00:06.70; speed 0.97 x real time
